<a href="https://colab.research.google.com/github/NelaNurazizah/ANTRIAN_MM2/blob/main/antrian_mm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("383aIxUOFG93RVsF58sjvgoSOqH_DXyX5e6oNu8ZAmPkxp9C")

In [ ]:
import os
if not os.path.exists('templates'):
    os.makedirs('templates')

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="id">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>M/M/2 Queuing Calculator</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f4f6f9; }
        .card { border: none; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }
        .header-title { color: #333; font-weight: 700; }
    </style>
</head>
<body>
    <div class="container mt-5">
        <div class="row justify-content-center">
            <div class="col-md-6">
                <div class="card p-4">
                    <h3 class="text-center header-title mb-4">M/M/2 Queue Calculator</h3>
                    <p class="text-center text-muted">Masukkan parameter waktu untuk menghitung kinerja antrian.</p>

                    {% with messages = get_flashed_messages(with_categories=true) %}
                        {% if messages %}
                            {% for category, message in messages %}
                                <div class="alert alert-{{ category }}">{{ message }}</div>
                            {% endfor %}
                        {% endif %}
                    {% endwith %}

                    <form method="POST" action="/">
                        <div class="mb-3">
                            <label for="inter_arrival_time" class="form-label">Waktu Antar Kedatangan (menit)</label>
                            <input type="number" step="0.01" class="form-control" id="inter_arrival_time" name="inter_arrival_time" placeholder="Contoh: 4" required>
                            <div class="form-text">Rata-rata waktu antara satu pelanggan dengan pelanggan berikutnya.</div>
                        </div>
                        <div class="mb-3">
                            <label for="service_time" class="form-label">Waktu Pelayanan per Server (menit)</label>
                            <input type="number" step="0.01" class="form-control" id="service_time" name="service_time" placeholder="Contoh: 3" required>
                            <div class="form-text">Rata-rata waktu yang dibutuhkan 1 pelayan untuk melayani 1 pelanggan.</div>
                        </div>
                        <div class="d-grid gap-2">
                            <button type="submit" class="btn btn-success btn-lg">Hitung Kalkulasi</button>
                        </div>
                    </form>
                </div>
            </div>
        </div>
    </div>
</body>
</html>

Writing templates/index.html


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="id">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hasil Perhitungan M/M/2</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">

    <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
    <script>
    window.MathJax = {
      tex: { inlineMath: [['$', '$'], ['\\(', '\\)']] }
    };
    </script>
    <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>

    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>

    <style>
        body { background-color: #f4f6f9; }
        /* Class untuk border warna-warni */
        .result-card { border-left: 5px solid #28a745; } /* HIJAU */
        .summary-card { border-left: 5px solid #28a745; } /* BIRU */
        .chart-card { border-left: 5px solid #28a745; }   /* MERAH */

        .step-box { background-color: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.05); margin-bottom: 20px; }
        h5 { font-weight: bold; }
        .h5-green { color: #28a745; }
        .h5-blue { color: #28a745; }
        .h5-red { color: #28a745; }
        .alert-info { border-left: 5px solid #0dcaf0; font-weight: 500; }
    </style>
</head>
<body>
    <div class="container mt-5 mb-5">
        <h2 class="text-center mb-4">Laporan Hasil Analisis Antrian (M/M/2)</h2>

        <div class="row justify-content-center">
            <div class="col-md-8">

                <div class="step-box result-card">
                    <h5 class="h5-green">1. Diketahui (Given)</h5>
                    <p>Parameter input dari sistem:</p>
                    <ul>
                        <li>Waktu antar kedatangan ($t_a$) = <strong>{{ res.input_arrival }} menit</strong></li>
                        <li>Waktu pelayanan ($t_s$) = <strong>{{ res.input_service }} menit</strong></li>
                        <li>Jumlah Pelayan ($c$) = <strong>2</strong></li>
                    </ul>
                </div>

                <div class="step-box result-card"> <h5 class="h5-green">2. Solusi Pengerjaan (Step-by-Step)</h5>

                    <p><strong>A. Menentukan Laju Kedatangan ($\lambda$)</strong></p>
                    <p>$$ \lambda = \frac{1}{t_a} = \frac{1}{ {{ res.input_arrival }} } $$</p>
                    <div class="alert alert-info">Laju kedatangan: <strong>{{ res.lambda }} pelanggan/menit</strong>.</div>
                    <hr>

                    <p><strong>B. Menentukan Laju Pelayanan ($\mu$)</strong></p>
                    <p>$$ \mu = \frac{1}{t_s} = \frac{1}{ {{ res.input_service }} } $$</p>
                    <div class="alert alert-info">Laju pelayanan: <strong>{{ res.mu }} pelanggan/menit</strong>.</div>
                    <hr>

                    <p><strong>C. Pemanfaatan Pelayan ($\rho$)</strong></p>
                    <p>$$ \rho = \frac{\lambda}{2\mu} = \frac{ {{ res.lambda }} }{ 2 \times {{ res.mu }} } = {{ res.rho }} $$</p>
                    <div class="alert alert-info">Tingkat kesibukan: <strong>{{ res.rho_percent }}%</strong>.</div>
                    <hr>

                    <p><strong>D. Waktu Rata-rata dalam Sistem ($W$)</strong></p>
                    <p>$$ W = \frac{1}{ {{ res.mu }} - \frac{ {{ res.lambda }} }{2} } = \frac{1}{ {{ res.calc_denominator_w }} } $$</p>
                    <div class="alert alert-info">Waktu dalam sistem: <strong>{{ res.w_system }} menit</strong>.</div>
                    <hr>

                    <p><strong>E. Waktu Rata-rata dalam Antrian ($W_q$)</strong></p>
                    <p>$$ W_q = \frac{ {{ res.lambda }}^2 }{ 2({{ res.mu }})({{ res.mu }} - \frac{ {{ res.lambda }} }{2}) } $$</p>
                    <div class="alert alert-info">Waktu dalam antrian: <strong>{{ res.w_queue }} menit</strong>.</div>
                </div>

                <div class="step-box summary-card">
                    <h5 class="h5-blue">3. Ringkasan Hasil Perhitungan</h5>
                    <table class="table table-bordered table-striped table-hover mt-3 text-center">
                        <thead class="table-primary">
                            <tr><th>Parameter</th><th>Simbol</th><th>Nilai</th><th>Satuan</th></tr>
                        </thead>
                        <tbody>
                            <tr><td class="text-start">Laju Kedatangan</td><td>$\lambda$</td><td><strong>{{ res.lambda }}</strong></td><td>Pelanggan/menit</td></tr>
                            <tr><td class="text-start">Laju Pelayanan</td><td>$\mu$</td><td><strong>{{ res.mu }}</strong></td><td>Pelanggan/menit</td></tr>
                            <tr><td class="text-start">Tingkat Kesibukan</td><td>$\rho$</td><td><strong>{{ res.rho }}</strong> ({{ res.rho_percent }}%)</td><td>-</td></tr>
                            <tr><td class="text-start">Waktu dalam Sistem</td><td>$W$</td><td><strong>{{ res.w_system }}</strong></td><td>Menit</td></tr>
                            <tr><td class="text-start">Waktu dalam Antrian</td><td>$W_q$</td><td><strong>{{ res.w_queue }}</strong></td><td>Menit</td></tr>
                        </tbody>
                    </table>
                </div>

                <div class="step-box chart-card">
                    <h5 class="h5-red">4. Grafik Simulasi (Analisis Sensitivitas)</h5>
                    <p class="text-muted small">Grafik ini menunjukkan prediksi: <em>"Berapa lama antrian (Wq) jika pelanggan yang datang semakin ramai (Lambda naik)?"</em></p>

                    <div style="position: relative; height:400px; width:100%">
                        <canvas id="queueChart"></canvas>
                    </div>
                </div>

                <div class="text-center">
                    <a href="/" class="btn btn-primary btn-lg mb-5">Hitung Skenario Lain</a>
                </div>

            </div>
        </div>
    </div>

    <script>
        const ctx = document.getElementById('queueChart').getContext('2d');

        // Menggunakan JSON.parse agar VS Code tidak menampilkan error palsu
        const labels = JSON.parse('{{ res.graph_labels | tojson }}');
        const dataPoints = JSON.parse('{{ res.graph_data | tojson }}');

        const userLambda = Number('{{ res.lambda }}');
        const userWq = Number('{{ res.w_queue }}');

        const chart = new Chart(ctx, {
            type: 'line',
            data: {
                labels: labels,
                datasets: [
                    {
                        label: 'Waktu Tunggu Antrian (Wq)',
                        data: dataPoints,
                        borderColor: 'rgba(220, 53, 69, 1)',
                        backgroundColor: 'rgba(220, 53, 69, 0.1)',
                        borderWidth: 2,
                        pointRadius: 1,
                        fill: true,
                        tension: 0.4
                    },
                    {
                        label: 'Posisi Anda Saat Ini',
                        data: [{x: userLambda, y: userWq}],
                        pointBackgroundColor: 'blue',
                        pointBorderColor: 'blue',
                        pointRadius: 8,
                        pointHoverRadius: 10,
                        showLine: false,
                        type: 'scatter'
                    }
                ]
            },
            options: {
                responsive: true,
                maintainAspectRatio: false,
                interaction: {
                    mode: 'index',
                    intersect: false,
                },
                scales: {
                    x: {
                        type: 'linear',
                        title: { display: true, text: 'Laju Kedatangan Pelanggan (λ)' },
                        ticks: { stepSize: 0.05 }
                    },
                    y: {
                        title: { display: true, text: 'Waktu Tunggu (Menit)' }
                    }
                },
                plugins: {
                    tooltip: {
                        callbacks: {
                            label: function(context) {
                                return context.dataset.label + ': ' + context.raw.y + ' menit';
                            }
                        }
                    }
                }
            }
        });
    </script>
</body>
</html>

Writing templates/result.html


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, flash, redirect, url_for
import json # Tambahan library untuk kirim data ke grafik

app = Flask(__name__)
app.secret_key = 'kunci_rahasia_untuk_flash_message'

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        try:
            # Ambil input
            inter_arrival_time = request.form.get('inter_arrival_time')
            service_time = request.form.get('service_time')

            if not inter_arrival_time or not service_time:
                flash('Semua kolom harus diisi!', 'danger')
                return redirect(url_for('index'))

            t_arrival = float(inter_arrival_time)
            t_service = float(service_time)

            if t_arrival <= 0 or t_service <= 0:
                flash('Nilai waktu harus lebih besar dari 0!', 'danger')
                return redirect(url_for('index'))

            # --- PERHITUNGAN UTAMA (M/M/2) ---
            arrival_rate = 1 / t_arrival # Lambda
            service_rate = 1 / t_service # Mu

            # Cek Stabilitas Awal
            if arrival_rate >= 2 * service_rate:
                 flash('Sistem tidak stabil! Laju kedatangan melebihi kapasitas 2 pelayan.', 'warning')
                 return redirect(url_for('index'))

            # Hitung Metrik Utama
            rho = arrival_rate / (2 * service_rate)
            denominator_w = service_rate - (arrival_rate / 2)
            w_system = 1 / denominator_w

            numerator_wq = arrival_rate ** 2
            denominator_wq = 2 * service_rate * (service_rate - (arrival_rate / 2))
            w_queue = numerator_wq / denominator_wq

            # --- PERHITUNGAN DATA SIMULASI (GRAFIK) ---
            # Kita buat simulasi: Apa yang terjadi jika Laju Kedatangan (Lambda) berubah?
            # Range simulasi: dari 0.01 sampai mendekati kapasitas maksimal (2 * Mu)
            graph_labels = [] # Sumbu X (Lambda)
            graph_data = []   # Sumbu Y (Wq)

            capacity_limit = 2 * service_rate
            max_sim_lambda = capacity_limit * 0.95 # Batasi sampai 95% kapasitas agar tidak infinity

            # Generate 50 titik data
            step = max_sim_lambda / 50
            current_lam = 0.01

            while current_lam <= max_sim_lambda:
                # Rumus Wq untuk M/M/2 dengan Lambda simulasi
                # Wq = lambda^2 / (2*mu * (mu - lambda/2))
                curr_den = 2 * service_rate * (service_rate - (current_lam / 2))
                if curr_den > 0:
                    curr_wq = (current_lam ** 2) / curr_den
                    graph_labels.append(round(current_lam, 3))
                    graph_data.append(round(curr_wq, 3))
                current_lam += step

            results = {
                'input_arrival': t_arrival,
                'input_service': t_service,
                'lambda': round(arrival_rate, 4),
                'mu': round(service_rate, 4),
                'rho': round(rho, 4),
                'rho_percent': round(rho * 100, 2),
                'w_system': round(w_system, 4),
                'w_queue': round(w_queue, 4),
                'calc_denominator_w': round(denominator_w, 4),
                'calc_num_wq': round(numerator_wq, 4),
                'calc_den_wq': round(denominator_wq, 4),
                # Data untuk Grafik
                'graph_labels': graph_labels,
                'graph_data': graph_data
            }

            return render_template('result.html', res=results)

        except ValueError:
            flash('Input harus berupa angka!', 'danger')
            return redirect(url_for('index'))

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

Writing app.py


In [ ]:
from pyngrok import ngrok

# Matikan tunnel lama jika ada
ngrok.kill()

# Buka tunnel baru di port 5000
public_url = ngrok.connect(5000)
print(f"🚀 Aplikasi berjalan di sini: {public_url}")

# Jalankan Flask
# NOTE: Di Colab, ini akan berjalan terus menerus (loading).
# Kalau mau stop, tekan tombol Stop (kotak) di samping sel.
!python app.py

🚀 Aplikasi berjalan di sini: NgrokTunnel: "https://facultative-nonhesitantly-bruce.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 462-506-135
127.0.0.1 - - [12/Jan/2026 04:15:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2026 04:15:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jan/2026 04:34:18] "POST / HTTP/1.1" 200 -
